In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pkl
import warnings
import time as time
import requests
from bs4 import BeautifulSoup
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from requests_html import HTMLSession,AsyncHTMLSession

In [2]:
data = pd.read_csv('/home/bh_parijat/data/main_dataset_update1.2.csv')

In [3]:
data.shape

(4791859, 21)

In [4]:
data.zipcode.value_counts()

11207.0    98484
10035.0    94918
11212.0    92070
10457.0    87027
10001.0    86859
           ...  
10152.0        4
10704.0        3
11020.0        2
10045.0        1
11559.0        1
Name: zipcode, Length: 214, dtype: int64

In [84]:
async def get_html_data(url = None):
    
    session = AsyncHTMLSession()

    response = await session.get(url)
    
    soup = BeautifulSoup(response.content,'html.parser')
    
    table_rows = soup.find_all('tr')
    
    return table_rows

In [85]:
url = "http://www.zipcodestogo.com/New%20York/"
table_rows = get_html_data(url = url)

/home/bh_parijat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: coroutine 'get_html_data' was never awaited
  


In [83]:
print(len(table_rows))

TypeError: object of type 'coroutine' has no len()

In [66]:
table_rows[-10].find_all('td')

[<td align="center"><a href="//www.zipcodestogo.com/Elmira/NY/14925/">14925</a></td>,
 <td>Elmira</td>,
 <td><a href="/Chemung/NY/">Chemung</a></td>,
 <td><div align="center"><a href="//maps.google.com/maps?oi=map&amp;q=14925">View Map</a></div></td>]

In [67]:
def create_map(rows = None):
    
    """
    Used for creating a map for zipcode to city and vice-versa
    
    """
    
    if not rows:
        return
    map_zipcode_city = {}
    
    map_city_zipcode = {}
    for row in rows:
        
        trs = row.find_all('td')
        
        if len(trs)!=4:
            continue
        zipcode = row.find_all('td')[0].text
        city = row.find_all('td')[1].text
        
        map_zipcode_city[zipcode] = city
        
        codes = map_city_zipcode.get(city,[])
        
        codes.append(zipcode)
        
        
    return map_city_zipcode,map_zipcode_city

In [68]:
map_city_zipcode,map_zipcode_city = create_map(rows=table_rows[3:])

In [69]:
len(map_zipcode_city.keys())

2205

In [77]:
def create_dataframe(map_zipcode_city=None):
    
    if map_zipcode_city is None:
        return None
    
    zipcode,city = [],[]
    for key in map_zipcode_city:
        zipcode.append(key)
        city.append(map_zipcode_city[key])
    zipcode_city_df = pd.DataFrame(data={'zipcode':zipcode,'city':city})
    
    return zipcode_city_df

In [78]:
zipcode_city_df = create_dataframe(map_zipcode_city=map_zipcode_city)
zipcode_city_df.head()

,zipcode,city
0,00501,Holtsville
1,00544,Holtsville
2,06390,Fishers Island
3,10001,New York
4,10002,New York
